<a href="https://colab.research.google.com/github/heghiw/lakovaci-linka/blob/main/experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Remove the existing directory
#!rm -rf lakovaci-linka

# Clone the repository again
!git clone https://github.com/heghiw/lakovaci-linka.git

# Navigate to the directory
%cd lakovaci-linka


Cloning into 'lakovaci-linka'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 25 (delta 7), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (25/25), 9.02 MiB | 32.53 MiB/s, done.
Resolving deltas: 100% (7/7), done.
/content/lakovaci-linka


data prep

In [9]:
import pandas as pd

# Define the path to the Excel file
file_path = '/content/lakovaci-linka/data1.xlsx'

# Read the sheet 'linka' into a DataFrame
linka_df = pd.read_excel(file_path, sheet_name='linka')

# Read the sheet 'recept' into a DataFrame
recept_df = pd.read_excel(file_path, sheet_name='recept')

print("Linka:")
print(linka_df)
print("\nRecepty:")
print(recept_df)



Linka:
             typ vany  id_vany   pozice_x_rel  pozice_x_cum
0               vstup         0           0.0           0.0
1        Teplý oplach         1        2300.0        2300.0
2             Postřik         2        1900.0        4200.0
3    Ponor odm železo         3        1900.0        6100.0
4    Ponor odm pozink         4        1800.0        7900.0
5            oplach 1         5        1800.0        9700.0
6            oplach 2         6        1800.0       11500.0
7       Moření železo         7        1800.0       13300.0
8       Moření pozink         8        1800.0       15100.0
9      Oplach moř žel         9        1800.0       16900.0
10  Oplach moř pozink        10        1800.0       18700.0
11    oplach společný        11        1800.0       20500.0
12           aktivace        12        1800.0       22300.0
13             fosfát        13        1800.0       24100.0
14             oplach        14        1800.0       25900.0
15            Oplach         21  

manipulator and vozik characteristics

In [5]:
manipulator_characteristics = {
    "okap_point": 2000,  # mm
    "highest_point": 2750,  # mm
    "deceleration_point": 500,  # mm before ponoreni
    "speed_after_ponoreni": 8,  # m/min
    "speed_going_up": 15,  # m/min
    "speed_going_down": 12,  # m/min
    "speed_left_right": 35  # m/min
}

vozik_characteristics = {
    "speed": 20  # m/min
}

In [13]:
print(manipulator_characteristics)

{'okap_point': 2000, 'highest_point': 2750, 'deceleration_point': 500, 'speed_after_ponoreni': 8, 'speed_going_up': 15, 'speed_going_down_before_deceleration': 12, 'speed_left_right': 35}


In [10]:
linka_df.columns = linka_df.columns.str.strip()
recept_df.columns = recept_df.columns.str.strip()
print(linka_df.columns)
print(recept_df.columns)

Index(['typ vany', 'id_vany', 'pozice_x_rel', 'pozice_x_cum'], dtype='object')
Index(['tech', 'id_vany', 'poradi_operace', 'cas_min', 'cas_max', 'cas_opt',
       'okap', 'okap_cas'],
      dtype='object')


ver3


In [31]:
import pandas as pd
import numpy as np

def path_opt(linka_df, recept_df, manipulator_characteristics, vozik_characteristics, num_manipulators=1, selected_tech='tech1'):
    # initialize outputs
    manipulator_utilization = {i: {'total_time': 0, 'idle_time': 0, 'busy_time': 0} for i in range(num_manipulators)}
    bath_utilization = {bath_id: {'total_time': 0, 'busy_time': 0} for bath_id in linka_df['id_vany'].unique()}
    movement_sequence = []

    # filter recipe for selected technology
    recipe = recept_df[recept_df['tech'] == selected_tech].sort_values('poradi_operace')

    # simulate movement for each item
    time = 0

    for _, step in recipe.iterrows():
        bath_id = step['id_vany']
        operation_time = step['cas_opt']
        okap_time = step['okap_cas'] if step['okap'] else 0

        if bath_id not in linka_df['id_vany'].values:
            print(f"Warning: Bath ID {bath_id} not found in linka_df. Skipping this step.")
            continue

        bath_x_pos = linka_df.loc[linka_df['id_vany'] == bath_id, 'pozice_x_cum'].values[0]

        # assign to manipulator that can minimize travel time
        manip_id = min(manipulator_utilization, key=lambda x: manipulator_utilization[x]['busy_time'] + abs(bath_x_pos - (movement_sequence[-1]['to'] if movement_sequence else linka_df.loc[linka_df['id_vany'] == linka_df['id_vany'].min(), 'pozice_x_cum'].values[0])))

        # movement time calculation
        if movement_sequence:
            last_x = movement_sequence[-1]['to']
        else:
            last_x = linka_df.loc[linka_df['id_vany'] == linka_df['id_vany'].min(), 'pozice_x_cum'].values[0]  # vstup

        travel_distance = abs(bath_x_pos - last_x)
        travel_time = travel_distance / (manipulator_characteristics['speed_left_right'] * 1000 / 60)

        # log movement
        movement_sequence.append({
            'time': time,
            'manipulator': manip_id,
            'from': last_x,
            'to': bath_x_pos,
            'travel_time': travel_time,
            'operation_time': operation_time,
            'okap_time': okap_time
        })

        # update time and utilization
        time += travel_time + operation_time + okap_time
        manipulator_utilization[manip_id]['total_time'] += travel_time + operation_time + okap_time
        manipulator_utilization[manip_id]['busy_time'] += travel_time + operation_time + okap_time
        manipulator_utilization[manip_id]['idle_time'] = time - manipulator_utilization[manip_id]['busy_time']
        bath_utilization[bath_id]['total_time'] += operation_time + okap_time
        bath_utilization[bath_id]['busy_time'] += operation_time + okap_time

    # Calculate utilization percentage
    total_process_time = sum(recipe['cas_opt'] + recipe['okap_cas'])
    for manip_id in manipulator_utilization:
        manipulator_utilization[manip_id]['utilization_percentage'] = (manipulator_utilization[manip_id]['busy_time'] / total_process_time) * 100

    for bath_id in bath_utilization:
        bath_utilization[bath_id]['utilization_percentage'] = (bath_utilization[bath_id]['busy_time'] / total_process_time) * 100

    return manipulator_utilization, bath_utilization, movement_sequence

# example usage
selected_tech = input("Enter the technology (e.g., tech1, tech2): ")
num_manipulators = int(input("Enter the number of manipulators: "))

manip_util, bath_util, movements = path_opt(linka_df, recept_df, manipulator_characteristics, vozik_characteristics, num_manipulators=num_manipulators, selected_tech=selected_tech)

# Convert utilization data to DataFrame for better visualization
manipulator_utilization_df = pd.DataFrame([{'Manipulator': manip_id, **util} for manip_id, util in manip_util.items()])
bath_utilization_df = pd.DataFrame([{'Bath ID': bath_id, **util} for bath_id, util in bath_util.items()])
movements_df = pd.DataFrame(movements)

print("Manipulator Utilization:")
print(manipulator_utilization_df)

print("\nBath Utilization:")
print(bath_utilization_df)

print("\nMovement Sequence:")
print(movements_df)



Enter the technology (e.g., tech1, tech2): tech1
Enter the number of manipulators: 6
Manipulator Utilization:
   Manipulator  total_time    idle_time   busy_time  utilization_percentage
0            0  446.057143  2314.657143  446.057143               16.563578
1            1  500.285714  1977.314286  500.285714               18.577264
2            2  360.285714  2404.371429  360.285714               13.378601
3            3  442.600000  2224.028571  442.600000               16.435202
4            4  512.342857  1328.971429  512.342857               19.024985
5            5  503.085714   709.714286  503.085714               18.681237

Bath Utilization:
    Bath ID  total_time  busy_time  utilization_percentage
0         0           0          0                0.000000
1         1         140        140                5.198663
2         2         160        160                5.941329
3         3         240        240                8.911994
4         4           0          0          

ver4 manipulators do not cross paths

In [43]:
import pandas as pd
import time as tm

def path_opt(linka_df, recept_df, manipulator_characteristics, num_manipulators=1, selected_tech='tech1', time_limit=300):
    # Initialize manipulator positions and task queue
    manipulator_positions = {i: 0 for i in range(num_manipulators)}
    movement_sequence = []

    # Filter recipe for selected technology
    recipe = recept_df[recept_df['tech'] == selected_tech].sort_values('poradi_operace') #all good

    # Simulate movement for each item, assuming continuous input at vstup
    start_time = tm.time()
    current_time = 0
    task_queue = [{'id_vany': 0, 'time': 0}]  # Start with an initial task at vstup

    last_sequence = None
    sequence_counter = 0

    while task_queue and (tm.time() - start_time) < time_limit:
        for manip_id in range(num_manipulators):
            if task_queue:
                # Get the next task from the queue
                task = task_queue.pop(0)
                bath_id = task['id_vany']
                bath_x_pos = linka_df.loc[linka_df['id_vany'] == bath_id, 'pozice_x_cum'].values[0]

                # Find the next operation for the current bath
                next_step = recipe.loc[recipe['id_vany'] == bath_id].iloc[0]
                operation_time = next_step['cas_opt']
                okap_time = next_step['okap_cas'] if next_step['okap'] else 0

                # Movement time calculation
                last_x = manipulator_positions[manip_id]
                travel_time = abs(bath_x_pos - last_x) / (manipulator_characteristics['speed_left_right'] * 1000 / 60)

                # Log movement
                movement_sequence.append({
                    'time': current_time,
                    'manipulator': manip_id,
                    'from': last_x,
                    'to': bath_x_pos,
                    'travel_time': travel_time,
                    'operation_time': operation_time,
                    'okap_time': okap_time
                })

                # Update time and manipulator position
                current_time += travel_time + operation_time + okap_time
                manipulator_positions[manip_id] = bath_x_pos

                # Add next task to the queue
                next_task_idx = recipe.index.get_loc(next_step.name) + 1
                if next_task_idx < len(recipe):
                    next_task = recipe.iloc[next_task_idx]
                    task_queue.append({'id_vany': next_task['id_vany'], 'time': current_time})

                # Check if the product needs to exit
                if bath_id == 100:
                    # Simulate new input at vstup
                    task_queue.append({'id_vany': 0, 'time': current_time})

                # Check for recurring sequence
                current_sequence = tuple(movement_sequence[-len(recipe):])
                if current_sequence == last_sequence:
                    sequence_counter += 1
                    if sequence_counter > 3:  # Arbitrary threshold for stability
                        print("Recurring sequence detected, stopping simulation.")
                        return movement_sequence, current_time / len(movement_sequence)
                else:
                    last_sequence = current_sequence
                    sequence_counter = 0

    # Calculate takt time
    takt_time = current_time / len(movement_sequence)

    return movement_sequence, takt_time

# Example usage
selected_tech = input("Enter the technology (e.g., tech1, tech2): ")
num_manipulators = int(input("Enter the number of manipulators: "))
time_limit = float(input("Enter the time limit in seconds: "))

movements, takt_time = path_opt(linka_df, recept_df, manipulator_characteristics, num_manipulators=num_manipulators, selected_tech=selected_tech, time_limit=time_limit)

# Convert movement data to DataFrame for better visualization
movements_df = pd.DataFrame(movements)

print("\nMovement Sequence:")
print(movements_df)

print("\nTakt Time:")
print(takt_time)



Enter the technology (e.g., tech1, tech2): tech1
Enter the number of manipulators: 8
Enter the time limit in seconds: 10

Movement Sequence:
               time  manipulator     from       to  travel_time  \
0      0.000000e+00            0      0.0      0.0     0.000000   
1      0.000000e+00            1      0.0   2300.0     3.942857   
2      1.439429e+02            2      0.0   4200.0     7.200000   
3      3.111429e+02            3      0.0   6100.0    10.457143   
4      5.616000e+02            4      0.0   9700.0    16.628571   
...             ...          ...      ...      ...          ...   
12315  2.107840e+06            3  16900.0  33600.0    28.628571   
12316  2.108109e+06            4  20500.0  35900.0    26.400000   
12317  2.108227e+06            5  22300.0  37700.0    26.400000   
12318  2.108344e+06            6  24100.0  39500.0    26.400000   
12319  2.108461e+06            7  25900.0  41800.0    27.257143   

       operation_time  okap_time  
0                  

incorporated both vertical and horuzontal movement for manipulator

In [15]:
import pandas as pd
import time as tm

def path_opt(linka_df, recept_df, manipulator_characteristics, num_manipulators=1, selected_tech='tech1', time_limit=300):
    # Initialize manipulator positions and task queue
    manipulator_positions = {i: {'x': 0, 'y': manipulator_characteristics['highest_point']} for i in range(num_manipulators)}
    movement_sequence = []

    # Filter recipe for selected technology
    recipe = recept_df[recept_df['tech'] == selected_tech].sort_values('poradi_operace')

    # Simulate movement for each item, assuming continuous input at vstup
    start_time = tm.time()
    current_time = 0
    task_queue = [{'id_vany': 0, 'time': 0}]  # Start with an initial task at vstup

    last_sequence = None
    sequence_counter = 0

    while task_queue and (tm.time() - start_time) < time_limit:
        for manip_id in range(num_manipulators):
            if task_queue:
                # Get the next task from the queue
                task = task_queue.pop(0)
                bath_id = task['id_vany']

                # Move to the bath first in the recipe
                bath_x_pos = linka_df.loc[linka_df['id_vany'] == bath_id, 'pozice_x_cum'].values[0]
                last_x = manipulator_positions[manip_id]['x']
                horizontal_travel_time = abs(bath_x_pos - last_x) / 1000 / manipulator_characteristics['speed_left_right'] * 60

                # Check if the position is occupied
                if bath_x_pos in [pos['x'] for pos in manipulator_positions.values() if pos['x'] != last_x]:
                    print(f"Manipulator {manip_id} cannot move to bath_x_pos: {bath_x_pos}, position occupied by another manipulator.")
                    task_queue.append(task)
                    continue

                # Log horizontal movement
                movement_sequence.append({
                    'time': current_time,
                    'manipulator': manip_id,
                    'from': last_x,
                    'to': bath_x_pos,
                    'horizontal_travel_time': horizontal_travel_time
                })
                current_time += horizontal_travel_time
                manipulator_positions[manip_id]['x'] = bath_x_pos

                # Go down to the bath
                y_distance_down = manipulator_characteristics['highest_point'] / 1000
                vertical_travel_time_down = y_distance_down / manipulator_characteristics['speed_going_down'] * 60
                current_time += vertical_travel_time_down

                # Change speed at 500 mm
                y_distance_slow_down = (manipulator_characteristics['highest_point'] - manipulator_characteristics['deceleration_point']) / 1000
                vertical_travel_time_slow_down = y_distance_slow_down / manipulator_characteristics['speed_after_ponoreni'] * 60
                current_time += vertical_travel_time_slow_down

                # Perform operation
                next_step = recipe.loc[recipe['id_vany'] == bath_id].iloc[0]
                operation_time = next_step['cas_opt']
                current_time += operation_time

                # Go up empty with speed 15
                vertical_travel_time_up = y_distance_down / manipulator_characteristics['speed_going_up'] * 60
                current_time += vertical_travel_time_up

                # Check okap
                okap_time = next_step['okap_cas'] if next_step['okap'] else 0
                if next_step['okap']:
                    # Stop for okap time
                    current_time += okap_time

                # Move to the next step in the recipe
                next_task_idx = recipe.index.get_loc(next_step.name) + 1
                if next_task_idx < len(recipe):
                    next_task = recipe.iloc[next_task_idx]
                    task_queue.append({'id_vany': next_task['id_vany'], 'time': current_time})

                # If finished, move to id_vany 100 to leave the finished product
                if bath_id == 100:
                    # Move to the exit position
                    exit_x_pos = linka_df.loc[linka_df['id_vany'] == 100, 'pozice_x_cum'].values[0]
                    horizontal_travel_time_to_exit = abs(exit_x_pos - bath_x_pos) / 1000 / manipulator_characteristics['speed_left_right'] * 60
                    current_time += horizontal_travel_time_to_exit

                    # Log movement to exit
                    movement_sequence.append({
                        'time': current_time,
                        'manipulator': manip_id,
                        'from': bath_x_pos,
                        'to': exit_x_pos,
                        'horizontal_travel_time': horizontal_travel_time_to_exit
                    })
                    manipulator_positions[manip_id]['x'] = exit_x_pos

                    # Simulate new input at vstup if the manipulator is empty
                    if manipulator_positions[manip_id]['x'] == exit_x_pos:
                        task_queue.append({'id_vany': 0, 'time': current_time})

                # Check for recurring sequence
                current_sequence = tuple(movement_sequence[-len(recipe):])
                if current_sequence == last_sequence:
                    sequence_counter += 1
                    if sequence_counter > 3:  # Arbitrary threshold for stability
                        print("Recurring sequence detected, stopping simulation.")
                        return movement_sequence, current_time / len(movement_sequence)
                else:
                    last_sequence = current_sequence
                    sequence_counter = 0

    # Calculate takt time
    takt_time = current_time / len(movement_sequence) if movement_sequence else 0

    return movement_sequence, takt_time

# Example usage
selected_tech = input("Enter the technology (e.g., tech1, tech2): ")
num_manipulators = int(input("Enter the number of manipulators: "))
time_limit = float(input("Enter the time limit in seconds: "))

movements, takt_time = path_opt(linka_df, recept_df, manipulator_characteristics, num_manipulators=num_manipulators, selected_tech=selected_tech, time_limit=time_limit)

# Convert movement data to DataFrame for better visualization
movements_df = pd.DataFrame(movements)

print("\nMovement Sequence:")
print(movements_df)

print("\nTakt Time:")
print(takt_time)


Enter the technology (e.g., tech1, tech2): tech1
Enter the number of manipulators: 8
Enter the time limit in seconds: 300

Movement Sequence:
                time  manipulator     from       to  horizontal_travel_time
0       0.000000e+00            0      0.0      0.0                0.000000
1       4.162500e+01            1      0.0   2300.0                3.942857
2       2.271929e+02            2      0.0   4200.0                7.200000
3       4.360179e+02            3      0.0   6100.0               10.457143
4       7.281000e+02            4      0.0   9700.0               16.628571
...              ...          ...      ...      ...                     ...
358557  7.266360e+07            3  33600.0   6100.0               47.142857
358558  7.266392e+07            4  35900.0   9700.0               44.914286
358559  7.266409e+07            5  37700.0  11500.0               44.914286
358560  7.266425e+07            6  39500.0  13300.0               44.914286
358561  7.266483e+07  

nejkratsi cesta

In [ ]:
import heapq
import pandas as pd

# Define the heuristic function (straight-line distance to goal)
def heuristic(current_positions, goal_positions, linka_df):
    total_heuristic = 0
    for current, goal in zip(current_positions, goal_positions):
        current_pos = linka_df.loc[linka_df['id_vany'] == current, 'pozice_x_cum'].values[0]
        goal_pos = linka_df.loc[linka_df['id_vany'] == goal, 'pozice_x_cum'].values[0]
        total_heuristic += abs(goal_pos - current_pos) / 1000  # Convert mm to m
    return total_heuristic

# Check if the state is valid (no two manipulators in the same position)
def is_valid_state(positions):
    return len(set(positions)) == len(positions)

# A* algorithm to find the shortest path for multiple manipulators
def a_star_multiple(graph, start_positions, goal_positions, linka_df, recept_df, num_manipulators, manipulator_characteristics, takt_limit):
    # Priority queue to store (total_cost, positions)
    queue = [(0, start_positions)]
    # Dictionary to store the minimum cost to reach each set of positions
    costs = {tuple(start_positions): 0}
    # Dictionary to store the path taken
    path = {tuple(start_positions): None}

    final_positions = None

    while queue:
        current_cost, current_positions = heapq.heappop(queue)

        # Check if all manipulators have reached their goal positions
        if all(pos == goal for pos, goal in zip(current_positions, goal_positions)):
            final_positions = tuple(current_positions)
            break

        for manip_id in range(num_manipulators):
            current_state = current_positions[manip_id]
            if current_state not in graph:
                continue  # Skip if no neighbors

            for neighbor in graph[current_state]:
                # Calculate horizontal movement cost
                horizontal_distance = abs(linka_df.loc[linka_df['id_vany'] == neighbor, 'pozice_x_cum'].values[0] -
                                          linka_df.loc[linka_df['id_vany'] == current_state, 'pozice_x_cum'].values[0]) / 1000
                horizontal_cost = horizontal_distance / manipulator_characteristics['speed_left_right']

                # Vertical movement cost (down and up can be handled by different manipulators)
                vertical_distance_down = manipulator_characteristics['highest_point'] / 1000
                vertical_cost_down = vertical_distance_down / manipulator_characteristics['speed_going_down']
                vertical_distance_slow_down = (manipulator_characteristics['highest_point'] - manipulator_characteristics['deceleration_point']) / 1000
                vertical_cost_slow_down = vertical_distance_slow_down / manipulator_characteristics['speed_after_ponoreni']
                vertical_cost_up = vertical_distance_down / manipulator_characteristics['speed_going_up']

                # Operation cost (convert from seconds to minutes)
                operation_time = recept_df.loc[recept_df['id_vany'] == neighbor, 'cas_opt'].values[0] / 60

                # Okap cost (convert from seconds to minutes)
                okap_time = (recept_df.loc[recept_df['id_vany'] == neighbor, 'okap_cas'].values[0] / 60) if recept_df.loc[recept_df['id_vany'] == neighbor, 'okap'].values[0] else 0

                # Total cost calculation
                total_cost = current_cost + horizontal_cost + vertical_cost_down + vertical_cost_slow_down + vertical_cost_up + operation_time + okap_time
                heuristic_cost = heuristic([neighbor], goal_positions, linka_df)
                total_cost_with_heuristic = total_cost + heuristic_cost

                # Update positions and costs
                new_positions = list(current_positions)
                new_positions[manip_id] = neighbor

                if is_valid_state(new_positions) and (tuple(new_positions) not in costs or total_cost_with_heuristic < costs[tuple(new_positions)]):
                    costs[tuple(new_positions)] = total_cost_with_heuristic
                    path[tuple(new_positions)] = current_positions
                    heapq.heappush(queue, (total_cost_with_heuristic, new_positions))

    if final_positions is None:
        print("No valid path found to goal positions.")
        print("Graph:", graph)
        return costs, path

    # Calculate celkový takt and positions
    celkovy_takt = costs.get(final_positions, float('inf'))
    if celkovy_takt > takt_limit:
        print(f"Warning: Celkový takt {celkovy_takt:.2f} minutes exceeds takt limit of {takt_limit} minutes.")
    else:
        print(f"Celkový takt: {celkovy_takt:.2f} minutes")

    # Print manipulator positions
    positions_table = []
    current_position = final_positions
    while current_position:
        positions_table.append(current_position)
        current_position = path.get(current_position)

    positions_table.reverse()
    print("Manipulator Positions:")
    for step, positions in enumerate(positions_table):
        print(f"Step {step}: {positions}")

    return costs, path

# Create the graph based on the Linka data for the selected technology
def create_graph(linka_df, recept_df, technology):
    graph = {}
    recipe_vany = recept_df[recept_df['tech'] == technology].sort_values('poradi_operace')['id_vany'].tolist()
    for i in range(len(recipe_vany) - 1):
        current_vana = recipe_vany[i]
        next_vana = recipe_vany[i + 1]
        if current_vana not in graph:
            graph[current_vana] = []
        graph[current_vana].append(next_vana)

        # Allow skipping baths
        for j in range(i + 2, len(recipe_vany)):
            graph[current_vana].append(recipe_vany[j])
    return graph



# Select technology
technology = 'tech1'  # Example: tech1, tech2, tech3

# Create the graph for the selected technology
graph = create_graph(linka_df, recept_df, technology)

# Define number of manipulators
num_manipulators = 3
# Start manipulators at different positions to avoid overlap
start_positions = [0, 1, 2]  # Example starting positions
goal_positions = [100, 100, 100]  # All manipulators aim to reach 'vystup'

# User-defined takt limit
takt_limit = 600  # Example takt limit in minutes

# Find shortest path for multiple manipulators
costs, path = a_star_multiple(graph, start_positions, goal_positions, linka_df, recept_df, num_manipulators, manipulator_characteristics, takt_limit)
